<a href="https://colab.research.google.com/github/romcanrom/DAP_rom/blob/main/DAP_project_interm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### **사용할 데이터 불러오기: pd.read_csv 이용**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving appstore_games.csv to appstore_games (3).csv


In [ ]:
df = pd.read_csv('appstore_games.csv')
df

,URL,ID,Name,Subtitle,Icon URL,Average User Rating,User Rating Count,Price,In-app Purchases,Description,Developer,Age Rating,Languages,Size,Primary Genre,Genres,Original Release Date,Current Version Release Date
0,https://apps.apple.com/us/app/sudoku/id284921427,284921427,Sudoku,NaN,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,4.0,3553.0,2.99,NaN,"Join over 21,000,000 of our fans and download ...",Mighty Mighty Good Games,4+,"DA, NL, EN, FI, FR, DE, IT, JA, KO, NB, PL, PT...",15853568.0,Games,"Games, Strategy, Puzzle",11/07/2008,30/05/2017
1,https://apps.apple.com/us/app/reversi/id284926400,284926400,Reversi,NaN,https://is4-ssl.mzstatic.com/image/thumb/Purpl...,3.5,284.0,1.99,NaN,"The classic game of Reversi, also known as Oth...",Kiss The Machine,4+,EN,12328960.0,Games,"Games, Strategy, Board",11/07/2008,17/05/2018
2,https://apps.apple.com/us/app/morocco/id284946595,284946595,Morocco,NaN,https://is5-ssl.mzstatic.com/image/thumb/Purpl...,3.0,8376.0,0.00,NaN,Play the classic strategy game Othello (also k...,Bayou Games,4+,EN,674816.0,Games,"Games, Board, Strategy",11/07/2008,5/09/2017
3,https://apps.apple.com/us/app/sudoku-free/id28...,285755462,Sudoku (Free),NaN,https://is3-ssl.mzstatic.com/image/thumb/Purpl...,3.5,190394.0,0.00,NaN,"Top 100 free app for over a year.\nRated ""Best...",Mighty Mighty Good Games,4+,"DA, NL, EN, FI, FR, DE, IT, JA, KO, NB, PL, PT...",21552128.0,Games,"Games, Strategy, Puzzle",23/07/2008,30/05/2017
4,https://apps.apple.com/us/app/senet-deluxe/id2...,285831220,Senet Deluxe,NaN,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,3.5,28.0,2.99,NaN,"""Senet Deluxe - The Ancient Game of Life and A...",RoGame Software,4+,"DA, NL, EN, FR, DE, EL, IT, JA, KO, NO, PT, RU...",34689024.0,Games,"Games, Strategy, Board, Education",18/07/2008,22/07/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17002,https://apps.apple.com/us/app/stack-puzzle-ris...,1474626442,Stack Puzzle : Rise Tower,"Blast the cubes, solve puzzle!",https://is5-ssl.mzstatic.com/image/thumb/Purpl...,NaN,NaN,0.00,NaN,"The goal is very simple, move the square horiz...",Zhigang Pei,4+,EN,64795648.0,Games,"Games, Entertainment, Casual, Strategy",30/07/2019,30/07/2019
17003,https://apps.apple.com/us/app/eachother/id1474...,1474919257,EachOther,NaN,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,NaN,NaN,0.00,NaN,Collect a score while you play!!\n\nBy linking...,Sultan Shindi,4+,EN,110341120.0,Games,"Games, Family, Strategy",1/08/2019,1/08/2019
17004,https://apps.apple.com/us/app/rabbit-vs-tortoi...,1474962324,Rabbit Vs Tortoise,NaN,https://is2-ssl.mzstatic.com/image/thumb/Purpl...,NaN,NaN,0.00,NaN,"""Rabbit Vs Tortoise is chess type cool simple ...",Vishal Baldha,4+,EN,23207936.0,Games,"Games, Strategy",3/08/2019,3/08/2019
17005,https://apps.apple.com/us/app/fatall/id1474963671,1474963671,FaTaLL,Most fun game!!!,https://is1-ssl.mzstatic.com/image/thumb/Purpl...,NaN,NaN,0.00,"9.99, 49.99, 3.99",Upgrade your character and use your skills to ...,Tayrem Games,4+,EN,196750336.0,Games,"Games, Strategy, Action",1/08/2019,1/08/2019


### **데이터 분석에 필요하지 않은 열 제거: drop**

In [ ]:
df.drop(['URL', 'ID', 'Subtitle', 'Icon URL', 'Description'], axis=1, inplace=True)
df

,Name,Average User Rating,User Rating Count,Price,In-app Purchases,Developer,Age Rating,Languages,Size,Primary Genre,Genres,Original Release Date,Current Version Release Date
0,Sudoku,4.0,3553.0,2.99,NaN,Mighty Mighty Good Games,4+,"DA, NL, EN, FI, FR, DE, IT, JA, KO, NB, PL, PT...",15853568.0,Games,"Games, Strategy, Puzzle",11/07/2008,30/05/2017
1,Reversi,3.5,284.0,1.99,NaN,Kiss The Machine,4+,EN,12328960.0,Games,"Games, Strategy, Board",11/07/2008,17/05/2018
2,Morocco,3.0,8376.0,0.00,NaN,Bayou Games,4+,EN,674816.0,Games,"Games, Board, Strategy",11/07/2008,5/09/2017
3,Sudoku (Free),3.5,190394.0,0.00,NaN,Mighty Mighty Good Games,4+,"DA, NL, EN, FI, FR, DE, IT, JA, KO, NB, PL, PT...",21552128.0,Games,"Games, Strategy, Puzzle",23/07/2008,30/05/2017
4,Senet Deluxe,3.5,28.0,2.99,NaN,RoGame Software,4+,"DA, NL, EN, FR, DE, EL, IT, JA, KO, NO, PT, RU...",34689024.0,Games,"Games, Strategy, Board, Education",18/07/2008,22/07/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17002,Stack Puzzle : Rise Tower,NaN,NaN,0.00,NaN,Zhigang Pei,4+,EN,64795648.0,Games,"Games, Entertainment, Casual, Strategy",30/07/2019,30/07/2019
17003,EachOther,NaN,NaN,0.00,NaN,Sultan Shindi,4+,EN,110341120.0,Games,"Games, Family, Strategy",1/08/2019,1/08/2019
17004,Rabbit Vs Tortoise,NaN,NaN,0.00,NaN,Vishal Baldha,4+,EN,23207936.0,Games,"Games, Strategy",3/08/2019,3/08/2019
17005,FaTaLL,NaN,NaN,0.00,"9.99, 49.99, 3.99",Tayrem Games,4+,EN,196750336.0,Games,"Games, Strategy, Action",1/08/2019,1/08/2019


In [ ]:
df.isnull().sum()

Name                               0
Average User Rating             9446
User Rating Count               9446
Price                             24
In-app Purchases                9324
Developer                          0
Age Rating                         0
Languages                         60
Size                               1
Primary Genre                      0
Genres                             0
Original Release Date              0
Current Version Release Date       0
dtype: int64

### **Handling Missing Value**



**missing value가 50% 이상 존재하는 Average User Rating, User Rating Count, In-app Purchase: 사용 시 null 데이터 삭제 후 이용**

**missing value가 적은 price, languages, size: 미리 null값을 drop, imputation으로 전처리 후 데이터 전체 사용**




**price: 앱의 결제 가격으로 모바일 앱 시장에서 유료 앱보다는 무료 앱이 다수인 점을 고려, median 대신 가장 많이 나타나는 값 mode를 이용해 imputation 적용**

In [ ]:
# Price 항목의 mode 값 확인
df['Price'].mode()

0    0.0
dtype: float64

In [ ]:
df.index[df['Price'].isnull()]

Int64Index([ 8341, 15216, 16104, 16158, 16351, 16358, 16628, 16635, 16715,
            16725, 16731, 16760, 16780, 16782, 16795, 16879, 16891, 16902,
            16942, 16951, 16978, 16980, 17000, 17001],
           dtype='int64')

In [ ]:
# Price 항목의 null 값을 Price 항목의 mode 값으로 대체함
most_freq = df['Price'].value_counts(dropna=True).idxmax()
df['Price'].fillna(most_freq, inplace = True)

In [ ]:
# Price의 missing value가 제대로 처리되었는지 확인
df['Price'].isnull().sum()

0

In [ ]:
df.isnull().sum()

Name                               0
Average User Rating             9446
User Rating Count               9446
Price                              0
In-app Purchases                9324
Developer                          0
Age Rating                         0
Languages                         60
Size                               1
Primary Genre                      0
Genres                             0
Original Release Date              0
Current Version Release Date       0
dtype: int64

**Languages 열을 first_lg, second_lg, others로 분할**

In [ ]:
# Languages의 missing value를 mode 값 'EN' 값으로 채워줌
df['Languages'] = df['Languages'].fillna(df['Languages'].mode()[0])

In [ ]:
# Languages에 missing value가 없음을 확인
df['Languages'].isnull().sum()

0

In [ ]:
# Languages 열을 first_lg, second_lg, others 세 개의 열로 나누어 제공하는 언어를 확인
df=df.join(df['Languages'].str.split(',', n=2, expand=True).rename(columns={0:'first_lg', 1:'second_lg', 2:'others'}))

In [ ]:
df['Genres'].unique()

array(['Games, Strategy, Puzzle', 'Games, Strategy, Board',
       'Games, Board, Strategy', ..., 'Games, Strategy, Sports, Music',
       'Social Networking, Games, Strategy, Casual',
       'Sports, Racing, Games, Strategy'], dtype=object)